In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Admin\\Projects\\ML Projects\\stock_sentiment\\notebooks'

In [3]:
os.chdir('../src')

In [4]:
%pwd

'c:\\Users\\Admin\\Projects\\ML Projects\\stock_sentiment\\src'

### Database setup

In [14]:
import mysql.connector
from mysql.connector import Error
import logging
import os
from dotenv import load_dotenv
load_dotenv()

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
def create_database():
    connection = mysql.connector.connect(
        host = 'localhost',
        user='root',
        password=os.getenv('MYSQL_PASSWORD')
    )
    cursor = connection.cursor()

    cursor.execute("CREATE DATABASE IF NOT EXISTS financial_news")
    logger.info("Database 'financial news' created")

    # create user
    cursor.execute("""
                CREATE USER IF NOT EXISTS 'finews_user'@'localhost'
                IDENTIFIED BY %s
                """, (os.getenv('DB_PASSWORD', 'password'),)
    )
    logger.info("User 'finews_user' created")
    cursor.execute("""
            GRANT ALL PRIVILEGES ON financial_news.* 
            TO 'finews_user'@'localhost'
        """)
    logger.info("Privileges granted")

    cursor.execute("FLUSH PRIVILEGES")

    logger.info("User 'finews_user' created with privileges")
        
    cursor.close()
    connection.close()
    return True

def create_tables():
    """Create all required tables"""
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='finews_user',
            password=os.getenv('DB_PASSWORD'),
            database='financial_news'
        )
        cursor = connection.cursor()
        
        # News articles table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS news_articles (
                id INT AUTO_INCREMENT PRIMARY KEY,
                article_id VARCHAR(255) UNIQUE NOT NULL,
                title TEXT NOT NULL,
                description TEXT,
                content TEXT,
                source VARCHAR(255),
                published_date DATETIME NOT NULL,
                url TEXT,
                category VARCHAR(100),
                country VARCHAR(10),
                language VARCHAR(10),
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                INDEX idx_published_date (published_date),
                INDEX idx_source (source),
                INDEX idx_category (category)
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci
        """)
        logger.info("✓ Table 'news_articles' created")
        
        # Sentiment analysis table
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS sentiment_analysis (
                id INT AUTO_INCREMENT PRIMARY KEY,
                article_id VARCHAR(255) NOT NULL,
                model_name VARCHAR(100) NOT NULL,
                sentiment VARCHAR(50) NOT NULL,
                confidence FLOAT,
                positive_score FLOAT,
                negative_score FLOAT,
                neutral_score FLOAT,
                analyzed_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                processing_time_ms FLOAT,
                FOREIGN KEY (article_id) REFERENCES news_articles(article_id)
                    ON DELETE CASCADE,
                INDEX idx_article_id (article_id),
                INDEX idx_model_name (model_name),
                INDEX idx_sentiment (sentiment),
                UNIQUE KEY unique_article_model (article_id, model_name)
            ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci
        """)
        logger.info("Table 'sentiment_analysis' created")
        
        connection.commit()
        cursor.close()
        connection.close()
        
        logger.info("All tables created successfully")
        return True
        
    except Error as e:
        logger.error(f"Table creation failed: {e}")
        return False

In [19]:
create_database()
create_tables()

INFO:__main__:Database 'financial news' created
INFO:__main__:User 'finews_user' created
INFO:__main__:Privileges granted
INFO:__main__:User 'finews_user' created with privileges
INFO:__main__:✓ Table 'news_articles' created
INFO:__main__:✓ Table 'sentiment_analysis' created
INFO:__main__:✓ All tables created successfully


True

### Database connection

In [20]:
from config.configuration import ConfigurationManager
from config_entity import DatabaseConfig
# from dotenv import load_dotenv
import mysql.connector
from mysql.connector import Error, pooling
import logging
from dataclasses import dataclass
logger = logging.getLogger(__name__)

In [21]:
class Database:
    def __init__(self, config=DatabaseConfig):
        self.config = config
        self.connection_pool = None
        
        if config.enabled:
            self._create_connection_pool()

    def _create_connection_pool(self):
        self.connection_pool = mysql.connector.pooling.MySQLConnectionPool(
            pool_name="finews_pool",
            pool_size=self.config.pool_size,
            pool_reset_session=True,
            host=self.config.host,
            port=self.config.port,
            database=self.config.database,
            user=self.config.user,
            password=self.config.password,
            charset=self.config.charset
        )
        logger.info("✓ MySQL connection pool created")

    def get_connection(self):
        if not self.config.enabled or not self.connection_pool:
            return None
        try:
            return self.connection_pool.get_connection()
        except Error as e:
            logger.error(f"Error getting connection: {e}")
            return None

In [22]:
config_manager = ConfigurationManager()
config_manager.config.database.password

'${MYSQL_PASSWORD}'

In [23]:
db_config = config_manager.get_database_config()
db_config

DatabaseConfig(enabled=True, type='mysql', host='localhost', port=3306, database='financial_news', user='finews_user', password='krishnadas1996', pool_size=5, charset='utf8mb4')

In [24]:
db_connection = Database(config=db_config)


INFO:__main__:✓ MySQL connection pool created


In [26]:
db_connection.get_connection()